## [--- Day 8: Playground ---](https://adventofcode.com/2025/day/8)

In [ ]:
class DSU:
    def __init__(self, num_points) -> None:
        self.parent = list(range(num_points))
        self.size = [1] * num_points
        self.num_circuits = num_points

    def find(self, i) -> int:
        if self.parent[i] == i:
            return i
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j) -> None:
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i != root_j:
            # Merge smaller into larger
            if self.size[root_i] < self.size[root_j]:
                root_i, root_j = root_j, root_i
            self.parent[root_j] = root_i
            self.size[root_i] += self.size[root_j]
            self.num_circuits -= 1

In [ ]:
import numpy as np
from scipy.spatial import cKDTree


def solve() -> None:
    with open("..\\data\\08.txt") as file:
        points = np.array([np.array([int(x), int(y), int(z)]) for x, y, z in (line.strip().split(',') for line in file.readlines())])
    
    tree = cKDTree(points)
    paired: set[int, int] = set()
    pairs: list[(int, float, int)] = []

    for i, point in enumerate(points):
        neighbors = tree.query(point, k=10)
        for distance, index in zip(neighbors[0][1:], neighbors[1][1:]):
            if (index, i) in paired: continue
    
            pairs.append((i, distance, index))
            paired.add((i, index))
    
    pairs.sort(key=lambda x: x[1])
    
    dsu = DSU(len(points))
    num_connections: int = 0
    connection_sizes: list[int] = []

    for i, _, j in pairs:
        dsu.union(i, j)
        num_connections += 1
        if num_connections == 1000:
            connection_sizes = [dsu.size[i] for i in range(len(points)) if dsu.parent[i] == i]
            connection_sizes.sort(reverse=True)
            result = connection_sizes[0] * connection_sizes[1] * connection_sizes[2]
        if dsu.num_circuits == 1:
            break

    if dsu.num_circuits > 1:
        print(f'{dsu.num_circuits - 1} circuits have not collapsed.')
        return

    print(f'Product of sizes of three largest circuits (part a): {result}')
    print(f'Product of last connection made (part b): {points[i][0] * points[j][0]}')


solve()

Product of sizes of three largest circuits (part a): 42315
Product of last connection made (part b): 8079278220
